In [ ]:
# === 1) Kütüphaneler + yollar ===
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Notebook "notebooks/" klasöründen çalışıyorsa kökü bir üst klasör olarak al
CWD = Path.cwd()
ROOT = CWD.parent if CWD.name == "notebooks" else CWD
ART = ROOT / "artifacts"

PARQUET_PATH = ART / "clean_dataset.parquet"     # asıl kaynak
CSV_PATH     = ART / "clean_dataset.csv"         # varsa alternatif
XLSX_PATH    = ART / "clean_dataset_new.xlsx"    # eski isim (varsa)
TARGET_COL   = "__target__TedaviSuresi"

print("CWD:", CWD)
print("ROOT:", ROOT)
print("Looking for:", PARQUET_PATH)

# === 2) Veriyi yükle (önce Parquet, sonra CSV, en son XLSX) ===
df = None
used = None
for p in [PARQUET_PATH, CSV_PATH, XLSX_PATH]:
    if p.exists():
        used = p
        if p.suffix == ".parquet":
            df = pd.read_parquet(p)
        elif p.suffix == ".csv":
            df = pd.read_csv(p)
        else:  # .xlsx / .xls
            df = pd.read_excel(p)
        break

if df is None:
    raise FileNotFoundError(
        "Dataset bulunamadı. Şu yollar denendi:\n"
        f"- {PARQUET_PATH}\n- {CSV_PATH}\n- {XLSX_PATH}"
    )

print("Loaded:", used)
print("Shape:", df.shape)
df.head()


In [ ]:
# === 3) Genel bilgi ve eksik değerler ===
df.info()
print("\nToplam eksik değer:", int(df.isnull().sum().sum()))

# === 4) Sayısal sütunlar için özet istatistikler ===
num_df = df.select_dtypes(include=[np.number])
desc = num_df.describe().T
print("\nSayısal sütun sayısı:", num_df.shape[1])
desc.head(10)


In [ ]:
# === 5) Hedef değişken dağılımı ===
if TARGET_COL in df.columns:
    counts = df[TARGET_COL].value_counts().sort_index()
    print("Hedef değişken dağılımı (ilk 20 değer):")
    display(counts.head(20))

    plt.figure(figsize=(8,4))
    plt.bar(counts.index.astype(str), counts.values)
    plt.title("Hedef Değişken Dağılımı (Tedavi Süresi)")
    plt.xlabel("Seans")
    plt.ylabel("Frekans")
    plt.xticks(rotation=45, ha="right")
    plt.tight_layout()
    plt.show()
else:
    print(f"⚠️ {TARGET_COL} bulunamadı!")


In [ ]:
# === 6) Yaş dağılımı ve hedef ile ilişkisi ===
if "Yas" in df.columns:
    # Histogram
    plt.figure(figsize=(6,4))
    plt.hist(df["Yas"].dropna(), bins=20, color="skyblue", edgecolor="black")
    plt.title("Yaş Dağılımı")
    plt.xlabel("Yaş")
    plt.ylabel("Frekans")
    plt.tight_layout()
    plt.show()

    # Scatter plot
    if TARGET_COL in df.columns:
        plt.figure(figsize=(6,4))
        plt.scatter(df["Yas"], df[TARGET_COL], s=10, alpha=0.6)
        plt.title("Yaş vs Tedavi Süresi")
        plt.xlabel("Yaş")
        plt.ylabel("Seans")
        plt.tight_layout()
        plt.show()
    else:
        print(f"⚠️ {TARGET_COL} bulunamadı (scatter çizilmedi).")
else:
    print("⚠️ Yas kolonu bulunamadı.")


In [ ]:
# === 7) Multi-label kolonlar için en sık görülen değerler ===
def top_n_binary(df, prefix, n=10):
    """Belirtilen prefix ile başlayan one-hot sütunların en çok görülen n tanesini gösterir."""
    cols = [c for c in df.columns if c.startswith(prefix + "__")]
    if not cols:
        print(f"⚠️ {prefix}: ilgili sütun bulunamadı."); 
        return
    
    sums = df[cols].sum().sort_values(ascending=False).head(n)
    print(f"\nTop {n} - {prefix}")
    display(sums.to_frame("count"))

    plt.figure(figsize=(8,4))
    plt.bar(sums.index.astype(str), sums.values, color="teal", edgecolor="black")
    plt.title(f"Top {n} {prefix}")
    plt.ylabel("Frekans")
    plt.xticks(rotation=45, ha="right")
    plt.tight_layout()
    plt.show()

# Her grup için çalıştır
for group in ["KronikHastalik", "Alerji", "Tanilar", "UygulamaYerleri"]:
    top_n_binary(df, group, n=10)



In [ ]:
# === 8) Tek-değerli kategorik değişkenlerin dağılımı ===
def plot_onehot_group(df, base, topn=10):
    """One-hot encode edilmiş tek değerli kategorik grupları özetler."""
    cols = [c for c in df.columns if c.startswith(base + "_")]
    if not cols:
        print(f"⚠️ {base}: ilgili sütun bulunamadı.")
        return
    
    sums = df[cols].sum().sort_values(ascending=False)
    print(f"\n{base} en sık görülen {topn} değer")
    display(sums.head(topn).to_frame("count"))

    plt.figure(figsize=(8,4))
    plt.bar(sums.head(topn).index.astype(str), sums.head(topn).values,
            color="orange", edgecolor="black")
    plt.title(f"{base} Dağılımı (Top {topn})")
    plt.ylabel("Frekans")
    plt.xticks(rotation=45, ha="right")
    plt.tight_layout()
    plt.show()

for base in ["Cinsiyet", "KanGrubu", "Uyruk", "Bolum", "TedaviAdi", "UygulamaSuresi"]:
    plot_onehot_group(df, base, topn=10)



In [ ]:
# === 9) EDA özetlerini CSV olarak kaydet ===
import os
from pathlib import Path

# artifacts/eda_outputs klasörünü hazırla
ROOT = Path.cwd().parent if Path.cwd().name == "notebooks" else Path.cwd()
out_dir = ROOT / "artifacts" / "eda_outputs"
out_dir.mkdir(parents=True, exist_ok=True)

# --- hedef değişken dağılımı ---
target_counts = df[TARGET_COL].value_counts().sort_index()
target_counts.to_csv(out_dir / "target_distribution.csv", header=["count"])
print("Hedef dağılım kaydedildi:", out_dir / "target_distribution.csv")

# --- multi-label top listeler ---
def save_top_list(df, prefix, n=20):
    cols = [c for c in df.columns if c.startswith(prefix + "__")]
    if cols:
        top = df[cols].sum().sort_values(ascending=False).head(n)
        top.to_csv(out_dir / f"{prefix}_top{n}.csv", header=["count"])
        print(f"{prefix} top{n} kaydedildi:", out_dir / f"{prefix}_top{n}.csv")
    else:
        print(f"⚠️ {prefix}: ilgili sütun bulunamadı, CSV üretilmedi.")

for p in ["KronikHastalik", "Alerji", "Tanilar", "UygulamaYerleri"]:
    save_top_list(df, p, n=20)

print("\n✅ EDA özetleri kaydedildi ->", out_dir)



In [ ]:
# === 10) Sayısal korelasyon ısı haritası + hedef korelasyonları ===
import numpy as np
import matplotlib.pyplot as plt

num_df = df.select_dtypes(include=[np.number])

# Büyük matrislerde görsellik zor olabilir; özetlemek için:
corr = num_df.corr(numeric_only=True)

plt.figure(figsize=(6,5))
plt.imshow(corr, cmap="viridis", aspect="auto")
plt.colorbar()
plt.title("Numeric correlation heatmap (overview)")
plt.tight_layout()
plt.show()

# Hedef ile korelasyonları (en yüksek | en düşük)
if TARGET_COL in corr.columns:
    corr_target = corr[TARGET_COL].drop(labels=[TARGET_COL], errors="ignore").sort_values(ascending=False)
    print("Top 15 positively correlated with target:")
    display(corr_target.head(15).to_frame("corr"))
    print("Top 15 negatively correlated with target:")
    display(corr_target.tail(15).to_frame("corr"))
else:
    print(f"⚠️ {TARGET_COL} bulunamadı, hedef korelasyonları hesaplanamadı.")
